In [2]:
import os
import cv2
import numpy as np
from tqdm import tqdm
import torch
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.modeling import build_model
from detectron2.checkpoint import DetectionCheckpointer
from detectron2.data import MetadataCatalog
from detectron2.structures import Instances
from detectron2.data.transforms import ResizeShortestEdge
from detectron2.model_zoo import get_config_file, get_checkpoint_url

def setup_model():
    cfg = get_cfg()
    cfg.merge_from_file(get_config_file("COCO-Detection/faster_rcnn_R_50_FPN_3x.yaml"))
    cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.5
    cfg.MODEL.WEIGHTS = get_checkpoint_url("COCO-Detection/faster_rcnn_R_50_FPN_3x.yaml")
    cfg.MODEL.DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
    model = build_model(cfg)
    model.eval()
    DetectionCheckpointer(model).load(cfg.MODEL.WEIGHTS)
    return model, cfg

# Hàm trích đặc trưng trung bình qua các ROI (hoặc top-1)
@torch.no_grad()
def extract_features(model, image_path, cfg):
    image = cv2.imread(image_path)
    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    height, width = image.shape[:2]

    aug = ResizeShortestEdge([640, 672, 704, 736, 768, 800], 1333)
    image = aug.get_transform(image).apply_image(image)
    image = torch.as_tensor(image.astype("float32").transpose(2, 0, 1))  # CHW
    inputs = {"image": image, "height": height, "width": width}
    
    features = model.backbone(image.unsqueeze(0).to(cfg.MODEL.DEVICE))
    proposal_generator = model.proposal_generator
    proposals, _ = proposal_generator([inputs], features)
    
    box_features = model.roi_heads._shared_roi_transform(
        [features[f] for f in model.roi_heads.in_features], [x.proposal_boxes for x in proposals]
    )
    box_pooled = box_features.mean(dim=0).cpu().numpy()
    
    return box_pooled

# Trích đặc trưng toàn bộ folder ảnh
def extract_all_features(img_dir, save_path="image_features.npy", save_ids="image_ids.npy", limit=None):
    model, cfg = setup_model()
    image_vectors = []
    image_ids = []

    files = sorted([f for f in os.listdir(img_dir) if f.endswith(".jpg")])
    if limit:
        files = files[:limit]

    for file in tqdm(files, desc="Extracting features"):
        img_path = os.path.join(img_dir, file)
        image_id = os.path.splitext(file)[0]
        try:
            vec = extract_features(model, img_path, cfg)
            image_vectors.append(vec)
            image_ids.append(image_id)
        except Exception as e:
            print(f"Failed {file}: {e}")
            continue

    np.save(save_path, np.array(image_vectors))
    np.save(save_ids, np.array(image_ids))
    print(f"Saved {len(image_ids)} features to {save_path}.")

# Ví dụ chạy:
if __name__ == "__main__":
    extract_all_features(
        img_dir="E:/Download/val2017",  # <== chỉnh đường dẫn tại đây
        save_path="image_features.npy",
        save_ids="image_ids.npy",
        limit=5000  # để test nhanh, bạn có thể bỏ giới hạn
    )

Extracting features:   0%|                                                            | 7/5000 [00:00<02:24, 34.51it/s]

Failed 000000000139.jpg: short_edge_length must be two values using 'range' sample style. Got [640, 672, 704, 736, 768, 800]!
Failed 000000000285.jpg: short_edge_length must be two values using 'range' sample style. Got [640, 672, 704, 736, 768, 800]!
Failed 000000000632.jpg: short_edge_length must be two values using 'range' sample style. Got [640, 672, 704, 736, 768, 800]!
Failed 000000000724.jpg: short_edge_length must be two values using 'range' sample style. Got [640, 672, 704, 736, 768, 800]!
Failed 000000000776.jpg: short_edge_length must be two values using 'range' sample style. Got [640, 672, 704, 736, 768, 800]!
Failed 000000000785.jpg: short_edge_length must be two values using 'range' sample style. Got [640, 672, 704, 736, 768, 800]!
Failed 000000000802.jpg: short_edge_length must be two values using 'range' sample style. Got [640, 672, 704, 736, 768, 800]!
Failed 000000000872.jpg: short_edge_length must be two values using 'range' sample style. Got [640, 672, 704, 736, 768

Extracting features:   0%|▏                                                          | 18/5000 [00:00<01:55, 42.99it/s]

Failed 000000001000.jpg: short_edge_length must be two values using 'range' sample style. Got [640, 672, 704, 736, 768, 800]!
Failed 000000001268.jpg: short_edge_length must be two values using 'range' sample style. Got [640, 672, 704, 736, 768, 800]!
Failed 000000001296.jpg: short_edge_length must be two values using 'range' sample style. Got [640, 672, 704, 736, 768, 800]!
Failed 000000001353.jpg: short_edge_length must be two values using 'range' sample style. Got [640, 672, 704, 736, 768, 800]!
Failed 000000001425.jpg: short_edge_length must be two values using 'range' sample style. Got [640, 672, 704, 736, 768, 800]!
Failed 000000001490.jpg: short_edge_length must be two values using 'range' sample style. Got [640, 672, 704, 736, 768, 800]!
Failed 000000001503.jpg: short_edge_length must be two values using 'range' sample style. Got [640, 672, 704, 736, 768, 800]!
Failed 000000001532.jpg: short_edge_length must be two values using 'range' sample style. Got [640, 672, 704, 736, 768

Extracting features:   1%|▎                                                          | 29/5000 [00:00<01:49, 45.56it/s]

...

Failed 000000579321.jpg: short_edge_length must be two values using 'range' sample style. Got [640, 672, 704, 736, 768, 800]!
Failed 000000579635.jpg: short_edge_length must be two values using 'range' sample style. Got [640, 672, 704, 736, 768, 800]!
Failed 000000579655.jpg: short_edge_length must be two values using 'range' sample style. Got [640, 672, 704, 736, 768, 800]!
Failed 000000579818.jpg: short_edge_length must be two values using 'range' sample style. Got [640, 672, 704, 736, 768, 800]!
Failed 000000579893.jpg: short_edge_length must be two values using 'range' sample style. Got [640, 672, 704, 736, 768, 800]!
Failed 000000579900.jpg: short_edge_length must be two values using 'range' sample style. Got [640, 672, 704, 736, 768, 800]!
Failed 000000579902.jpg: short_edge_length must be two values using 'range' sample style. Got [640, 672, 704, 736, 768, 800]!
Failed 000000579970.jpg: short_edge_length must be two values using 'range' sample style. Got [640, 672, 704, 736, 768

Extracting features: 100%|████████████████████████████████████████████████████████▉| 4994/5000 [02:20<00:00, 37.51it/s]

Failed 000000580197.jpg: short_edge_length must be two values using 'range' sample style. Got [640, 672, 704, 736, 768, 800]!
Failed 000000580294.jpg: short_edge_length must be two values using 'range' sample style. Got [640, 672, 704, 736, 768, 800]!
Failed 000000580410.jpg: short_edge_length must be two values using 'range' sample style. Got [640, 672, 704, 736, 768, 800]!
Failed 000000580418.jpg: short_edge_length must be two values using 'range' sample style. Got [640, 672, 704, 736, 768, 800]!
Failed 000000580757.jpg: short_edge_length must be two values using 'range' sample style. Got [640, 672, 704, 736, 768, 800]!
Failed 000000581062.jpg: short_edge_length must be two values using 'range' sample style. Got [640, 672, 704, 736, 768, 800]!
Failed 000000581100.jpg: short_edge_length must be two values using 'range' sample style. Got [640, 672, 704, 736, 768, 800]!
Failed 000000581206.jpg: short_edge_length must be two values using 'range' sample style. Got [640, 672, 704, 736, 768

Extracting features: 100%|█████████████████████████████████████████████████████████| 5000/5000 [02:20<00:00, 35.61it/s]

Failed 000000581317.jpg: short_edge_length must be two values using 'range' sample style. Got [640, 672, 704, 736, 768, 800]!
Failed 000000581357.jpg: short_edge_length must be two values using 'range' sample style. Got [640, 672, 704, 736, 768, 800]!
Failed 000000581482.jpg: short_edge_length must be two values using 'range' sample style. Got [640, 672, 704, 736, 768, 800]!
Failed 000000581615.jpg: short_edge_length must be two values using 'range' sample style. Got [640, 672, 704, 736, 768, 800]!
Failed 000000581781.jpg: short_edge_length must be two values using 'range' sample style. Got [640, 672, 704, 736, 768, 800]!
Saved 0 features to image_features.npy.
